In [1]:
import pandas as pd

In [2]:
from conllu import parse

In [ ]:
# path = 
# text_file_name = 

In [3]:
df = pd.read_excel('../data/data_pisat_k_Dat.xlsx')

In [8]:
df.head()

,Left context,Center,Punct,Right context,Created,Full context
0,о котором я уже,писал,,"в статье "" Мастер к",2004,"Это делают мастерицы московского клуба "" Красн..."
1,Он,писал,,"о любви к Алевтине, о",1998,"Он писал о любви к Алевтине, о том, что не при..."
2,"не только анималист, он",пишет,,"пейзажи, делает иллюстрации (",1987,"Олег Цингер не только анималист, он пишет пейз..."
3,замечательно не только личностью,пишущего,",","его горячим чувством к Фоме,",1987,Письмо француза замечательно не только личност...
4,"этой мизерабельной своей наружности говорил,",писал,",","горевал!) ― а какой это,",1923,"Ведь вот, и наружность, пожалуй, чиновничья, «..."


In [5]:
df.shape[0]

610

In [6]:
df.columns

Index(['Reversed left context', 'Reversed center', 'Left context', 'Center',
       'Punct', 'Right context', 'Title', 'Author', 'Birthday', 'Header',
       'Created', 'Sphere', 'Type', 'Topic', 'Publication', 'Publ_year',
       'Medium', 'Ambiguity', 'Full context'],
      dtype='object')

In [7]:
df.drop(columns=['Reversed left context', 'Reversed center', 'Title', 'Author', 'Birthday', 'Header', 'Publ_year', 
             'Medium', 'Ambiguity', 'Publication', 'Sphere', 'Type', 'Topic'], inplace = True)

In [9]:
df['Created'].describe()

count      610
unique     230
top       1999
freq        16
Name: Created, dtype: object

In [10]:
with open('sents_pisat_k_Dat.txt', 'a', encoding = "UTF-8") as f:
    for i in range(df.shape[0]):
        sent = df['Full context'][i]
        f.write(sent[:sent.find('[')-1] + '\n')

In [29]:
!udpipe --tokenize --tag --parse\
    ../models/russian-syntagrus-ud-2.0-170801.udpipe sents_pisat_k_Dat.txt > parsed_sents_pisat_k_Dat.conllu

Loading UDPipe model: done.


In [16]:
with open('parsed_sents_pisat_k_Dat.conllu', 'r', encoding = "UTF-8") as f:
    parsed_sents_pisat_k_Dat = f.read()

## Парсим предложения для питона

In [17]:
sentences = parse(parsed_sents_pisat_k_Dat)

In [64]:
sentences[6][5]['feats']

OrderedDict([('Animacy', 'Anim'),
             ('Case', 'Dat'),
             ('Gender', 'Fem'),
             ('Number', 'Sing')])

In [65]:
object_form = {'Case': 'Dat', 'Animacy': 'Anim'}
verb = 'писать'
preposition = 'к'

In [88]:
def check_object(token, object_form):
    if 'Foreign' in token['feats'] and token['feats']['Foreign'] == 'Yes':
        return False
    for feature in object_form:
        try:
            if token['feats'][feature] != object_form[feature]:
                return False
        except KeyError:
            print(token)
    return True

In [89]:
def find_triplet_in_sentence(sent, verb, object_form, preposition=None):
    # TODO: подумать про случаи, когда несколько подходящих глаголов в предложении
    triplet = {}
    verb_id = None
    object_id = None
    for token in sent:
        
        # ищем id глагола, чтобы потом искать его зависимые
        if token['lemma'] == verb:
            verb_id = token['id']
            triplet['verb'] = token['form']
            
        # ищем зависимые (NB: в нашем случае зависимые всегда идут после глагола, поэтому в одном цикле)
        if token['head'] == verb_id:
            if token['upostag'] in ['NOUN', 'PROPN'] and check_object(token, object_form):
                object_id = token['id']
                triplet['object'] = token['form']
                break # мы нашли объект (TODO: bla bla)
                
    if preposition: # looking for a preposition, if there is
        for token in sent:
            if token['head'] == object_id and token['form'] == preposition:
                triplet['preposition'] = preposition
           
    return triplet

In [96]:
find_triplet_in_sentence(sentences[5], 'писать', object_form, preposition='к')

{'verb': 'писали', 'object': 'человеку', 'preposition': 'к'}

In [100]:
def get_all_triples(sentences, verb, object_form, preposition=None):
    triples = []
    for sent in sentences:
        triples.append(find_triplet_in_sentence(sent, verb, object_form, preposition))
    return triples

In [101]:
triples = get_all_triples(sentences, verb, object_form, preposition='к')

In [102]:
triples[:10]

[{'verb': 'писал'},
 {'verb': 'писал'},
 {'verb': 'пишет'},
 {},
 {},
 {'verb': 'писали', 'object': 'человеку', 'preposition': 'к'},
 {'verb': 'писать', 'object': 'Анне', 'preposition': 'к'},
 {'verb': 'писал'},
 {'verb': 'пишет'},
 {'verb': 'писал'}]

## Пример
касаться до gen vs. касаться gen (от касаться зависит NOUN/PROPN + case-Gen (?) и не зависит до)